In [2]:
import seaborn as sb
import sys  
import pandas as pd
import requests
import numpy as np
import datetime
import time
import pickle
import matplotlib.pyplot as plt
import time
%matplotlib inline

from bs4 import BeautifulSoup as bs
import urllib

In [149]:
from bs4 import BeautifulSoup
import re
import string
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.preprocessing import LabelEncoder
from scipy.sparse import csr_matrix, hstack
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from scipy import sparse
from sklearn.preprocessing import StandardScaler
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import accuracy_score
from sklearn.svm import SVR
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from random import sample 

from tqdm import tqdm
import nltk
from pymorphy2 import MorphAnalyzer
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\viktoria.sinditskaya\AppData\Roaming\nltk_dat
[nltk_data]     a...
[nltk_data]   Package stopwords is already up-to-date!


True

In [148]:
def load_obj(name):
    with open(name + '.pkl', 'rb') as f:
        return pickle.load(f)

def save_obj(obj, name):
    with open(name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

In [150]:
df = load_obj('data_clean_df')

In [152]:
df.head()

,about,activities,age_cat,bdate,books,can_access_closed,city,country,education_form,education_status,faculty_name,followers_count,games,graduation,has_mobile,home_town,id,interests,is_closed,movies,music,occupation,quotes,relation,sex,track_code,university_name,verified,political,langs,religion,inspired_by,people_main,life_main,smoking,alcohol,schools_name,schools_grad_year,has_facebook,has_instagram,has_livejournal,has_twitter,has_skype,has_relative
1,,актриса,30_40,14.11.1984,,True,Moscow,Russia,,,,939222,,0,1,Москва,45269508,,False,,,university,,0,1,b2842eb9f_rndxNFGDYLHVpek0wfI0VPeH7_L2UIFO-0uq...,ГИТИС,1,Не указано,Не указано,Не указано,Не указано,Не указано,,Не указано,Не указано,,,0,0,0,1,0,1
4,,,30_40,12.4,,True,Moscow,Russia,,,,536139,,-1,1,,314833033,,False,,,work,,-1,1,434bbf0afpE6Ldd9Egf_m5ep42e8CuSMprLVXk-3KngDNZ...,,1,Не указано,Не указано,Не указано,Не указано,Не указано,,Не указано,Не указано,,,0,0,0,0,0,1
5,🏷Концерты (букинг) +7 985 226 7676\n🛍Реклама/P...,Лена Темникова — популярная российская певица,30_40,18.4,,True,Moscow,Russia,,,,528028,,0,1,,243556640,,False,,,work,,4,1,d41b43eaWtMJH9O5e15g3fJZXHfTvmlXuJZi5BDHDqmFUD...,,1,Не указано,[Русский],Не указано,Не указано,Не указано,,Не указано,Не указано,,,0,1,0,0,0,1
7,По вопросам организации концертов:\n\nПо вопро...,"певица, актриса, ведущая",30_40,1.3,"Ги де Мопассан, Конфуций, Борис Акунин",True,Moscow,Russia,,,Факультет физической культуры,450821,,0,1,Москва,164542053,,False,"""Служебный роман"", ""Зеленая миля"", ""Человек, к...","Beyonce, Shakira, Алла Пугачева, Уматурман",Не указано,,0,1,58518e8cqBV_N0KD2JTXIVRWQxh6zZwmEzAV2OmZuWBKdK...,МГАФК (Мосспортакадемия),1,Не указано,[Русский],Не указано,Не указано,Не указано,,Не указано,Не указано,,,0,0,0,0,0,1
9,,,30_40,12.5.1987,,True,Moscow,Russia,,,,376624,,-1,1,,81133993,,False,,,work,,-1,1,97803050Z599F6OhYv1wh-5wesMApIgG1QQW0dhxiKXNPz...,,1,Не указано,Не указано,Не указано,Не указано,Не указано,,Не указано,Не указано,,,0,0,0,1,0,1


In [153]:
numrc_col = [x for x in df.columns if df[x].dtype == 'float64' or df[x].dtype == 'int64']

In [155]:
df[numrc_col].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 860 entries, 1 to 3895
Data columns (total 13 columns):
followers_count    860 non-null int64
graduation         860 non-null int64
has_mobile         860 non-null int64
id                 860 non-null int64
relation           860 non-null int64
sex                860 non-null int64
verified           860 non-null int64
has_facebook       860 non-null int64
has_instagram      860 non-null int64
has_livejournal    860 non-null int64
has_twitter        860 non-null int64
has_skype          860 non-null int64
has_relative       860 non-null int64
dtypes: int64(13)
memory usage: 94.1 KB


In [157]:
str_col = [x for x in df.columns if df[x].dtype == 'object']

In [162]:
wall = load_obj('users_wall_posts')

In [163]:
len(wall)

482

In [165]:
chrs_to_delete = string.punctuation + u'»' + u'«' + u'—' + u'“' + u'„' + u'•' + u'#'
translation_table = {ord(c): None for c in chrs_to_delete if c != u'-'}
units = MorphAnalyzer.DEFAULT_UNITS
morph = MorphAnalyzer(result_type=None, units=units)
PortSt = PorterStemmer()
stopw = set(
    [w for w in stopwords.words(['russian', 'english'])]
    + [u'это', u'году', u'года', u'также', u'етот',
    u'которые', u'который', u'которая', u'поэтому',
    u'весь', u'свой', u'мочь', u'eтот', u'например',
    u'какой-то', u'кто-то', u'самый', u'очень', u'несколько',
    u'источник', u'стать', u'время', u'пока', u'однако',
    u'около', u'немного', u'кроме', u'гораздо', u'каждый',
    u'первый', u'вполне', u'из-за', u'из-под',
    u'второй', u'нужно', u'нужный', u'просто', u'большой',
    u'хороший', u'хотеть', u'начать', u'должный', u'новый', u'день',
    u'метр', u'получить', u'далее', u'именно', u'апрель',
    u'сообщать', u'разный', u'говорить', u'делать',
    u'появиться', u'2016',
    u'2015', u'получить', u'иметь', u'составить', u'дать', u'читать',
    u'ничто', u'достаточно', u'использовать',
    u'принять', u'практически',
    u'находиться', u'месяц', u'достаточно', u'что-то', u'часто',
    u'хотеть', u'начаться', u'делать', u'событие', u'составлять',
    u'остаться', u'заявить', u'сделать', u'дело',
    u'примерно', u'попасть', u'хотя', u'лишь', u'первое',
    u'больший', u'решить', u'число', u'идти', u'давать', u'вопрос',
    u'сегодня', u'часть', u'высокий', u'главный', u'случай', u'место',
    u'конец', u'работать', u'работа', u'слово', u'важный', u'сказать']
)
url_start = 'http[s]?://'
url_end = (
    '(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
)
pattern = url_start + url_end

In [168]:
users_clean_wall_dict = {}
counter = 0

for user_id, doc in wall.items():
    try:
        soup = BeautifulSoup(doc, 'html.parser')
        body = ' '.join(
            [tag.string.replace('\\n', ' ').replace('\\r', ' ')
            for tag in soup.descendants if tag.string]
        )
        body = re.sub('\[.*?\]','', body)
        body = re.sub(pattern,'', body)
    except Exception as ex:

        print('Unable to parse posts of user {}'.format(user_id))
    if body != '':
        body_clean = body.translate(translation_table).lower().strip()
        #токенизируем текст (грубо говоря, разбиваем на слова)
        words = word_tokenize(body_clean)
        tokens = []
        # stemming and text normalization
        for word in words:
            if re.match('^[a-z0-9-]+$', word) is not None:
                #если токен попадает под паттерн английского слова или цифры - делаем стемминг
                tokens.append(PortSt.stem(word))
            elif word.count('-') > 1:
                #если токен представляет собой набор символов --- , которые мы не удаляли, так как он может входить в слово ефис, просто добавляем это токен целиком
                tokens.append(word)
            else:
                #если токен русское слово - применяем к нему нормальную форму
                normal_forms =morph.normal_forms(word)
                tokens.append(normal_forms[0] if normal_forms else word)
        # remove stopwords and leave unique words only
        tokens = filter(
            lambda token: token not in stopw, sorted(set(tokens))
        )

        # leave all words with more than 3 chars
        tokens = filter(lambda token: len(token) > 3, tokens)
    else:
        tokens = []
    counter += 1
    if counter % 100 == 0:
        print("{0} docs processed".format(counter))
    #словарь, где id - это id группы, значения - ее очищенные токенизированные слова
    users_clean_wall_dict[user_id] = tokens
#словарь id группы: очищенные слова по ее постам
user_clean_doc_dict = {key: list(val) for key, val in users_clean_wall_dict.items()}


100 docs processed
200 docs processed
300 docs processed
400 docs processed


In [172]:
def add_post(x):
    uid = x['id']
    try:
            post = user_clean_doc_dict[uid]
            return post
    except Exception:
            return 


In [173]:
df['post'] = df.apply(lambda x:add_post(x),axis=1)

# Подготовка данных для модели

Для начала попробуем добавить дополнительные столбцы, которые могут помочь в определении возраста

Используем данные из столбца дата год окончания высшего учебного заведения и год окончания школы, чтобы как то определить примерный возраст

In [190]:
df.loc[df['schools_grad_year'] == '','schools_grad_year'] = -1

C:\Users\viktoria.sinditskaya\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [203]:
def get_avg_age_from_grad_dates(row):
    grad_school = int(row['schools_grad_year'])
    grad_uni = int(row['graduation'])
    years = []
    if grad_uni > 0:
        for year in range(21, 27):
            years.append(year + 2019 - grad_uni)
    if grad_school > 0:
        for year in range(17, 20):
            years.append(year + 2019 - grad_school)
    if len(years) != 0:
        return int(np.mean(years))
    else:
        return 0
    

In [204]:
df['guessed_age'] = df.apply(get_avg_age_from_grad_dates, axis=1)

C:\Users\viktoria.sinditskaya\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [206]:
df['guessed_age'].head()

22     0
36     0
74     0
77     0
139    0
Name: guessed_age, dtype: int64

In [207]:
df.head()

,about,activities,age_cat,bdate,books,can_access_closed,city,country,education_form,education_status,faculty_name,followers_count,games,graduation,has_mobile,home_town,id,interests,is_closed,movies,music,occupation,quotes,relation,sex,track_code,university_name,verified,political,langs,religion,inspired_by,people_main,life_main,smoking,alcohol,schools_name,schools_grad_year,has_facebook,has_instagram,has_livejournal,has_twitter,has_skype,has_relative,post,guessed_age
22,,Эзотерика,30_40,18.9.1988,*Анастасия-Влади́мир Мегре́ * Портрет Дориана ...,True,Tallinn,Estonia,,,,223979,,0,1,Раквере,156432930,"фотография, Эзотерика",False,,,Не указано,"“In order to experience everyday spirituality,...",0,1,8b0ec723Vrcmg8oRplL3-ldLEyUBaAFrvjMXAgXUmhl1l2...,,1,Не указано,"[English, Eesti, Русский]",Не указано,Не указано,Не указано,,Не указано,Не указано,,-1,0,0,0,0,0,1,"[автор, администратор, адрес, бессонный, битва...",0
36,,,30_40,8.2.1989,,True,Dnipropetrovsk (Dnipro),Ukraine,,,,232554,,-1,1,,9562528,,False,,,university,,-1,1,47a7d04a8ec6VLZTq2TqO_uuQOgIlk85KWcC6wsrDcmVci...,,0,Не указано,Не указано,Не указано,Не указано,Не указано,,Не указано,Не указано,,-1,1,1,0,0,0,1,"[100500, 27🎂всё, abscheck, beautyть, center, c...",0
74,,,30_40,14.3,,True,Moscow,Russia,,,,50692,,-1,1,,2801899,,False,,,Не указано,,-1,1,d83fb197braO8DtNw7wLF_vs7bPMHstqwSkUnCP-VzE2jP...,,1,Не указано,Не указано,Не указано,Не указано,Не указано,,Не указано,Не указано,,-1,0,0,0,0,0,1,"[13-ий, 1530, 2017, 2018, 2100, 212vipwildpart...",0
77,\n\nМОЯ ГРУППА:\nhttp://vkontakte.ru/club1291...,Импрессионистическая,30_40,,Научные,True,Kyiv,Ukraine,,,,49489,Карточные,0,1,Тель-Авив,12113139,Вдохновляющие,False,Исторические,Душевная,Не указано,,0,1,6be575180lQ2eXcG9dVYd9z_K6keeWMhxiTkZOi8MTaV1q...,,1,3,"[Русский, עברית, English, Українська]",Не указано,"Все, что вокруг",6,,Не указано,Не указано,,-1,0,1,0,0,0,1,"[1085, 38098, 6-го, annaalkhim, artikselfmad, ...",0
139,,Актриса,30_40,26.4.1989,"Секрет, трансерфинг реальности, психология, би...",True,Moscow,Russia,,,,16689,,0,1,,468500574,"Кино, Музыка, спорт",False,,,work,,1,1,96e7d5f9fAd7zOsx_4EDw4lvOeSlrmSSEfKd9PmxqPJ-ao...,,1,Не указано,[Русский],Не указано,Не указано,Не указано,,Не указано,Не указано,,-1,0,0,0,0,0,1,"[-быть, 1200-2100, 1600, 2000, 2011, 2018, 201...",0


Попробуем теперь извлечь данные из даты рождения

In [213]:
def get_age_from_brthd(val):
    if val != '':
        if len(val.split('.')) == 3:
            return 2019 - int(val[-4:])
    return 0

In [214]:
df['guessed_age'] = df['bdate'].apply(get_age_from_brthd)

C:\Users\viktoria.sinditskaya\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


Обработаем переменную типа занятости как указатель возраста: чем старше тип занятости - тем выше ранг переменной

In [220]:
rang_occupation = {'Не указано' : 0, 'work' : 3, 'university' : 2, 'school' : 1}
df['occupation'].replace(rang_occupation, inplace=True)

C:\Users\viktoria.sinditskaya\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\generic.py:6586: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [221]:
df['occupation'].value_counts()

0    168
3     99
2     71
1     18
Name: occupation, dtype: int64

Теперь попробуем ввести некоторые бинарные признаки, которые будут характеризовать общую заполненность профиля. Исходим из предположения, что у более взрослых пользователей профиль менее заполнен. Выделим признаки, по которым будем смотреть заполнение и переменную, которая будет характеризовать степень заполненности

In [216]:
df.columns

Index(['about', 'activities', 'age_cat', 'bdate', 'books', 'can_access_closed',
       'city', 'country', 'education_form', 'education_status', 'faculty_name',
       'followers_count', 'games', 'graduation', 'has_mobile', 'home_town',
       'id', 'interests', 'is_closed', 'movies', 'music', 'occupation',
       'quotes', 'relation', 'sex', 'track_code', 'university_name',
       'verified', 'political', 'langs', 'religion', 'inspired_by',
       'people_main', 'life_main', 'smoking', 'alcohol', 'schools_name',
       'schools_grad_year', 'has_facebook', 'has_instagram', 'has_livejournal',
       'has_twitter', 'has_skype', 'has_relative', 'post', 'guessed_age'],
      dtype='object')

In [217]:
#признаки для анализа
feat_completeness_lst = ['about', 'activities', 'books','games','has_mobile', 'home_town','interests','movies', 'music', 'quotes', 'relation', 'political', 'langs', 'religion', 'inspired_by',
       'people_main', 'life_main', 'smoking', 'alcohol','has_facebook', 'has_instagram', 'has_livejournal',
       'has_twitter', 'has_skype', 'has_relative']

In [222]:
df[ ['about', 'activities', 'books','games','has_mobile', 'home_town','interests','movies', 'music', 'quotes', 'relation', 'political', 'langs', 'religion', 'inspired_by',
       'people_main', 'life_main', 'smoking', 'alcohol','has_facebook', 'has_instagram', 'has_livejournal',
       'has_twitter', 'has_skype', 'has_relative']]

,about,activities,books,games,has_mobile,home_town,interests,movies,music,quotes,relation,political,langs,religion,inspired_by,people_main,life_main,smoking,alcohol,has_facebook,has_instagram,has_livejournal,has_twitter,has_skype,has_relative
22,,Эзотерика,*Анастасия-Влади́мир Мегре́ * Портрет Дориана ...,,1,Раквере,"фотография, Эзотерика",,,"“In order to experience everyday spirituality,...",0,Не указано,"[English, Eesti, Русский]",Не указано,Не указано,Не указано,,Не указано,Не указано,0,0,0,0,0,1
36,,,,,1,,,,,,-1,Не указано,Не указано,Не указано,Не указано,Не указано,,Не указано,Не указано,1,1,0,0,0,1
74,,,,,1,,,,,,-1,Не указано,Не указано,Не указано,Не указано,Не указано,,Не указано,Не указано,0,0,0,0,0,1
77,\n\nМОЯ ГРУППА:\nhttp://vkontakte.ru/club1291...,Импрессионистическая,Научные,Карточные,1,Тель-Авив,Вдохновляющие,Исторические,Душевная,,0,3,"[Русский, עברית, English, Українська]",Не указано,"Все, что вокруг",6,,Не указано,Не указано,0,1,0,0,0,1
139,,Актриса,"Секрет, трансерфинг реальности, психология, би...",,1,,"Кино, Музыка, спорт",,,,1,Не указано,[Русский],Не указано,Не указано,Не указано,,Не указано,Не указано,0,0,0,0,0,1
171,***************************************** \n...,Мечтать...Жить..и..Любить,"Преступление и наказание, Мастер и Маргарита, ...",собственная судьба и чужие чувства ********...,1,Санкт-Петербург,Солнце...море...пляж...ходить по магазинам...м...,"Сладкий ноябрь, Прости за любовь, If only/Если...",,"♥ Нет ничего не справедливей, чем скучать по к...",1,Не указано,[Русский],Верю в себя,Не указано,Не указано,,Не указано,Не указано,0,0,0,0,0,1
962,● Арт-директор\nart-director@nyusha.ru\n\n● PR...,,,,1,Москва,,,,,0,3,[Русский],Не указано,Не указано,Не указано,,Не указано,Не указано,0,0,0,0,0,1
964,http://instagram.com/kristydobro\n\n__________...,,,,1,Москва,,,,,0,Не указано,[Русский],Не указано,Не указано,Не указано,,Не указано,Не указано,0,1,0,1,0,1
971,YouTube блогер Elli Di\nДобропорядочный =),"Блогер Elli Di, помогаю понять собак и других ...","Робинзон Крузо, Три Мушкетера, Граф Монте-Крис...","The Sims 2, The Sims 3, The Sims 4 =) Call Of ...",1,Москва,"YouTube, SMM, SEO... Путешествия, яхтинг, вока...","Незнайка на луне, Один дома, терминал, Превосх...","Red Hot Chili Peppers, Metallica, Oliver Kolet...",,4,Не указано,"[Русский, English]",Не указано,Не указано,Не указано,,Не указано,Не указано,1,0,0,0,0,1
978,"У меня только 1 аккаунт, остальные ФЕЙКИ! не в...",,,,1,Санкт-Петербург,,,,,6,Не указано,"[Русский, English, Français]",Не указано,Не указано,Не указано,,Не указано,Не указано,0,0,0,0,1,1


In [226]:
def get_avg_completeness(row):
    counter = 0
    #отдельно обрабатываем те столбцы, где отсутствие - это пустая строка или  "Не указано"
    for col in ['about', 'activities', 'books','games','home_town','interests','movies', 'music', 'quotes' ,'political', 'langs', 'religion', 'inspired_by',
       'people_main', 'life_main', 'smoking', 'alcohol']:
        if df[col] != '' and df[col] != 'Не указано':
            counter += 1
    #отдельно обрабатываем числовые признаки
    for col in ['has_mobile','political','relation','has_facebook', 'has_instagram', 'has_livejournal',
       'has_twitter', 'has_skype', 'has_relative']:
        if col != -1 and col != 0:
            counter += 1
    return np.round(counter/len(feat_completeness_lst),2)

In [228]:
df['avg_completeness'] = df.apply(lambda x: get_avg_completeness(x),axis=1)

C:\Users\viktoria.sinditskaya\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


Теперь отдельно выделим длину поста и количество эмоджи в постах в предположении того, что у более молодого поколения больше эмоджи и больше длина постов

In [245]:
import regex
import re

# общее число символов в постах
df['post_length'] = df['post'].apply(lambda text: len(text))

# общее число тэгов в постах
def count_tags(text):
    text = ' '.join(text)
    return len(regex.findall('#', text))

df['tags_number'] = df['post'].apply(lambda x: count_tags(x))

# общее число эмоджи в постах
import emoji

def count_emoji(text):
    text = ' '.join(text)
    emoji_list = []
    data = regex.findall(r'\X', text)
    for word in data:
        if any(char in emoji.UNICODE_EMOJI for char in word):
            emoji_list.append(word)

    return len(emoji_list)

df['emoji_number'] = df['post'].apply(count_emoji)

C:\Users\viktoria.sinditskaya\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
C:\Users\viktoria.sinditskaya\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':
C:\Users\viktoria.sinditskaya\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .lo

In [425]:
df['guessed_age'].idxmax()

22

In [246]:
df.head()

,about,activities,age_cat,bdate,books,can_access_closed,city,country,education_form,education_status,faculty_name,followers_count,games,graduation,has_mobile,home_town,id,interests,is_closed,movies,music,occupation,quotes,relation,sex,track_code,university_name,verified,political,langs,religion,inspired_by,people_main,life_main,smoking,alcohol,schools_name,schools_grad_year,has_facebook,has_instagram,has_livejournal,has_twitter,has_skype,has_relative,post,guessed_age,avg_completeness,post_length,tags_number,emoji_number
22,,Эзотерика,30_40,18.9.1988,*Анастасия-Влади́мир Мегре́ * Портрет Дориана ...,True,Tallinn,Estonia,,,,223979,,0,1,Раквере,156432930,"фотография, Эзотерика",False,,,0,"“In order to experience everyday spirituality,...",0,1,8b0ec723Vrcmg8oRplL3-ldLEyUBaAFrvjMXAgXUmhl1l2...,,1,Не указано,"[English, Eesti, Русский]",Не указано,Не указано,Не указано,,Не указано,Не указано,,-1,0,0,0,0,0,1,"[автор, администратор, адрес, бессонный, битва...",31,0.60,156,0,0
36,,,30_40,8.2.1989,,True,Dnipropetrovsk (Dnipro),Ukraine,,,,232554,,-1,1,,9562528,,False,,,2,,-1,1,47a7d04a8ec6VLZTq2TqO_uuQOgIlk85KWcC6wsrDcmVci...,,0,Не указано,Не указано,Не указано,Не указано,Не указано,,Не указано,Не указано,,-1,1,1,0,0,0,1,"[100500, 27🎂всё, abscheck, beautyть, center, c...",30,0.36,429,0,31
74,,,30_40,14.3,,True,Moscow,Russia,,,,50692,,-1,1,,2801899,,False,,,0,,-1,1,d83fb197braO8DtNw7wLF_vs7bPMHstqwSkUnCP-VzE2jP...,,1,Не указано,Не указано,Не указано,Не указано,Не указано,,Не указано,Не указано,,-1,0,0,0,0,0,1,"[13-ий, 1530, 2017, 2018, 2100, 212vipwildpart...",0,0.36,633,0,173
77,\n\nМОЯ ГРУППА:\nhttp://vkontakte.ru/club1291...,Импрессионистическая,30_40,,Научные,True,Kyiv,Ukraine,,,,49489,Карточные,0,1,Тель-Авив,12113139,Вдохновляющие,False,Исторические,Душевная,0,,0,1,6be575180lQ2eXcG9dVYd9z_K6keeWMhxiTkZOi8MTaV1q...,,1,3,"[Русский, עברית, English, Українська]",Не указано,"Все, что вокруг",6,,Не указано,Не указано,,-1,0,1,0,0,0,1,"[1085, 38098, 6-го, annaalkhim, artikselfmad, ...",0,0.84,585,0,137
139,,Актриса,30_40,26.4.1989,"Секрет, трансерфинг реальности, психология, би...",True,Moscow,Russia,,,,16689,,0,1,,468500574,"Кино, Музыка, спорт",False,,,3,,1,1,96e7d5f9fAd7zOsx_4EDw4lvOeSlrmSSEfKd9PmxqPJ-ao...,,1,Не указано,[Русский],Не указано,Не указано,Не указано,,Не указано,Не указано,,-1,0,0,0,0,0,1,"[-быть, 1200-2100, 1600, 2000, 2011, 2018, 201...",30,0.52,1117,0,285


Готово, теперь оставим только те признаки, которые нам нужны

In [248]:
df = df[['age_cat','followers_count','sex','post','guessed_age','avg_completeness','post_length','tags_number','emoji_number']].copy()

Начинаем готовить данные к моделированию

In [253]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = le.fit_transform(df['age_cat'])

In [256]:
df.drop('age_cat', axis=1, inplace=True)

In [257]:
from sklearn.model_selection import train_test_split

In [435]:
X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=0.2,  shuffle=True, stratify=y, random_state=121)

Будем пробовать  LogisticRegression c перебором n_gram_range. Отдельно попробуем CountVectorizer и FTIdfVectorizer

In [436]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [437]:
def make_post_for_model(df_train, df_test, feature_post, params,CountVect=True ):
    if CountVect:
        vect = CountVectorizer()
        vect.set_params(**params)

        X_tfidf_train = vect.fit_transform(df_train[feature_post])
        X_tfidf_test = vect.transform(df_test[feature_post])
    else:
        vect = TfidfVectorizer()
        vect.set_params(**params)
        
        X_tfidf_train = vect.fit_transform(df_train[feature_post])
        X_tfidf_test = vect.transform(df_test[feature_post])

    return X_tfidf_train, X_tfidf_test
    

In [438]:
from sklearn.preprocessing import StandardScaler

In [439]:
def prepare_model(df_train, df_test, post_train, post_test):
    scaler = StandardScaler()
    df_train_for_model = df_train.drop('post',axis=1)
    df_test_for_model = df_test.drop('post',axis=1)

    X_train_short = pd.DataFrame(data=scaler.fit_transform(df_train_for_model.values), 
                                     index=df_train_for_model.index, 
                                     columns=df_train_for_model.columns)

    X_test_short = pd.DataFrame(data=scaler.transform(df_test_for_model.values), 
                                     index=df_test_for_model.index, 
                                     columns=df_test_for_model.columns)

    X_train_short_sparse = sparse.csr_matrix(X_train_short)
    X_test_short_sparse = sparse.csr_matrix(X_test_short)

    x_train = hstack([X_train_short_sparse, post_train])
    x_test = hstack([X_test_short_sparse, post_test])
    
    return x_train, x_test

Попробуем биграммы юниграммы триграммы для каждого из векторайзера

In [440]:
X_train.loc[:,'post'] = X_train['post'].apply(lambda x: ' '.join(x))
X_test.loc[:,'post'] = X_test['post'].apply(lambda x: ' '.join(x))

C:\Users\viktoria.sinditskaya\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [441]:
from sklearn.metrics import roc_auc_score

In [442]:
from sklearn.preprocessing import  LabelBinarizer

In [443]:
def run_model(model, x_train, x_test, y_train, y_test):
    model.fit(x_train, y_train)
    y_pred_proba = model.predict_proba(x_test)
    y_pred =  model.predict(x_test)
    lb = LabelBinarizer()
    lb.fit(y_test)
    y_test_n = lb.transform(y_test)
    y_pred_n = lb.transform(y_pred)
    return (model,roc_auc_score(y_test_n,y_pred_n,average='micro'),accuracy_score(y_test,y_pred))

In [444]:

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

Результаты будем записывать в таблицу

In [445]:
table_results = pd.DataFrame(columns=['model_name', 'ROC-AUC' ,
                                     'Accuracy, test'])

In [532]:

def model_maker(x_train, x_test, y_train, y_test, df_for_results):
    lst_vectorizers = []
    for ngram in [(1,1),(1,2),(1,3)]:
        for bols in [True,False]:
            X_vect_train, X_vect_test = make_post_for_model(x_train, x_test, 'post', {'ngram_range':ngram, 'analyzer':'word', 'max_features':10000},bols )
            lst_vectorizers.append((X_vect_train, X_vect_test))
            name =  "logistic ngram range:{},CountVectorizer is {}".format(ngram,bols)
            df_for_results = df_for_results.append({'model_name' :name,'ROC-AUC':None ,
                                         'Accuracy, test' : None},ignore_index=True)
            
    models_lst = []
    model = LogisticRegression()
    for num,i in enumerate(lst_vectorizers):
        X_vect_train, X_vect_test = i[0],i[1]
        x_train, x_test = prepare_model(X_train, X_test, X_vect_train, X_vect_test )
        model, auc, accuracy = run_model(model, x_train, x_test, y_train, y_test)
        models_lst.append(model)
        df_for_results.loc[num,'ROC-AUC'] = auc
        df_for_results['ROC-AUC'] = df_for_results['ROC-AUC'].astype('float')
        df_for_results.loc[num,'Accuracy, test'] = accuracy
    best_model_index = df_for_results['ROC-AUC'].idxmax()
    #TODO сохранить табличку с резульататми, а затем протранкейтить 
    return models_lst[best_model_index]
        
        

In [533]:
best_model = model_maker(X_train, X_test, y_train, y_test, table_results)

In [513]:
table_results.truncate(before=-1, after=-1)

,model_name,ROC-AUC,"Accuracy, test"
